In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("../input/hackerearth-effectiveness-of-std-drugs/dataset/train.csv")
test = pd.read_csv("../input/hackerearth-effectiveness-of-std-drugs/dataset/test.csv")

In [ ]:
train.head(3)

In [ ]:
train['year'] = pd.DatetimeIndex(train['drug_approved_by_UIC']).year
train['month'] = pd.DatetimeIndex(train['drug_approved_by_UIC']).month
train["approved_time"] = train['year']



for i in range(len(train)):
    a=2020-train["year"].iloc[i]
    b=train["month"].iloc[i]/12
    train["approved_time"].iloc[i] = a+b
    
train = train.drop(["drug_approved_by_UIC","year","month"],axis=1)



In [ ]:
test['year'] = pd.DatetimeIndex(test['drug_approved_by_UIC']).year
test['month'] = pd.DatetimeIndex(test['drug_approved_by_UIC']).month
test["approved_time"] = test['year']

for i in range(len(test)):
    a=2020-test["year"].iloc[i]
    b=test["month"].iloc[i]/12
    test["approved_time"].iloc[i] = a+b
    
test = test.drop(["drug_approved_by_UIC","year","month"],axis=1)


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train["name_of_drug"]  = le.fit_transform(train["name_of_drug"])
train["use_case_for_drug"]  = le.fit_transform(train["use_case_for_drug"])
test["name_of_drug"]  = le.fit_transform(test["name_of_drug"])
test["use_case_for_drug"]  = le.fit_transform(test["use_case_for_drug"])

In [ ]:
x=train
x=x.drop(["patient_id","review_by_patient","base_score","name_of_drug","use_case_for_drug"],axis=1)
y=train["base_score"]

x_test = test
x_testf = x_test.drop(["patient_id","review_by_patient","name_of_drug","use_case_for_drug"],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.1, random_state=42)

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(x, label=y)
dtest = xgb.DMatrix(x_test, label=y_test)
param = {'max_depth':4 , 'eta': 0.15, 'objective': 'reg:squarederror','lambda':1.5}
param['nthread'] = 15
param['eval_metric'] = 'rmse'

evallist = [(dtest, 'eval'), (dtrain, 'train')]

num_round = 4000
bst = xgb.train(param, dtrain, num_round, evallist)

#ypred = bst.predict(x_testf)

In [ ]:
dtest = xgb.DMatrix(x_testf)
y_pred = bst.predict(dtest)

In [ ]:
from lightgbm import LGBMRegressor

reg = LGBMRegressor(boosting_type='dart', num_leaves=30, max_depth=-1, learning_rate=1, n_estimators=200,reg_alpha=0.005, reg_lambda=0.5)
reg.fit(x_train,y_train)

y_pred = reg.predict(x_test)
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

y_pred = reg.predict(x_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
  
 # create regressor object 
regressor = RandomForestRegressor(n_estimators =200 , random_state = 5,min_samples_leaf=1,oob_score = True,min_samples_split=2) 
  
# fit the regressor with x and y data 
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

In [ ]:
from sklearn.svm import SVR

svm = SVR(C=2.0, epsilon=0.2)
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)

In [ ]:
from catboost import Pool, CatBoostRegressor
train_pool = Pool(x_train, 
                  y_train
                  )
eval_set = Pool(x_test, 
                  y_test
                  )
test_pool = Pool(x_testf
                 ) 

model = CatBoostRegressor(iterations=4000, 
                          depth=6, 
                          learning_rate=0.5, 
                          loss_function='RMSE',
                         bagging_temperature= 5,
                         border_count=600,
                         leaf_estimation_iterations=2)
model.fit(train_pool)

test_pool = Pool(x_testf
                 )
y_pred = model.predict(test_pool)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, preds)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
def norm(x):
  return (x - x['mean']) / x['std']
xnn = norm(x)


def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(x.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()

EPOCHS = 300

history = model.fit(
  x, y,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "mse")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')


In [ ]:
new = pd.DataFrame(test.patient_id)
new["base_score"] = y_pred
testf = new.to_csv(r'C:\Users\Naman Paharia\Desktop\testfin2.csv', index = None, header=True)
